In [5]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import bqport
from bqport import PositionType
import importlib
import scoring_model

importlib.reload(scoring_model)

bq = bql.Service()
d = bq.data
f = bq.func

In [6]:
def Calc_Inv_BETA(index, asset_list, cur_date, rotation, cur):
    
    start_date = cur_date - datetime.timedelta(weeks = 4*rotation)
    end_date = cur_date
    series =  d.day_to_day_total_return(start = start_date, end = end_date, fill = 'prev', currency = cur)
    bench_returns = f.value(series, bq.univ.list(index))
    inv_beta = 1 / f.slope(bench_returns, series)

    request = bql.Request(asset_list, {'INV_RAW_BETA': inv_beta})

    res = bq.execute(request)
    Inv_Betas=res[0].df()
    return Inv_Betas['INV_RAW_BETA']

def Calc_Weights(w):
    w=np.array(w)
    weight = w/w.sum()
    return weight
    
def Equal_Weight(n):
    w = [1/n]*n
    return w

In [7]:
#INPUTS    ********************
index = 'SXXP Index'
cur = 'EUR'
factor = 'valoracion' #'calidad', 'momentum', 'riesgo', 'valoracion'
rotation = 6 #months
n_assets = 20
equal_weight = True
start_date = datetime.datetime(2005, 1, 1) #YYYY,m,d
end_date = datetime.datetime(2018, 7, 1) #YYYY,m,d

In [8]:
###############    MAIN    ###############

aux_date = start_date
while end_date >= aux_date:
    print(aux_date)
    
    raw_data = scoring_model.get_data(aux_date.strftime('%Y-%m-%d'), cur, [index], True)
    data = raw_data[['ID',factor]]
    data = data.sort_values(by = [factor], ascending = False).head(n_assets)

    asset_list = data['ID'].tolist()

    if equal_weight == True:
        weights = Equal_Weight(n_assets)
    else:
        Inv_Betas = Calc_Inv_BETA(index, asset_list, aux_date, rotation, cur)
        weights = Calc_Weights(Inv_Betas)
    
    if aux_date == start_date:
        port_df = pd.DataFrame({'date': aux_date,'security': asset_list, 'weight': weights})
    else :
        aux_port_df = pd.DataFrame({'date': aux_date,'security': asset_list, 'weight': weights})
        port_df = pd.concat([port_df, aux_port_df])

    aux_date = aux_date + relativedelta(months=rotation)

port_df.set_index(['date', 'security'], inplace = True)
portfolio_obj = bqport.new_portfolio(from_ = port_df, currency = cur, type_ = PositionType.DRIFTING_WEIGHT, name = 'BQNT_WF_' + index.replace(' Index','') + '_' + factor + '_' + str(n_assets))
portfolio_obj.save()
print('Exportado a PRTU')

2005-01-01 00:00:00
2005-07-01 00:00:00
2006-01-01 00:00:00
2006-07-01 00:00:00
2007-01-01 00:00:00
2007-07-01 00:00:00
2008-01-01 00:00:00
2008-07-01 00:00:00
2009-01-01 00:00:00
2009-07-01 00:00:00
2010-01-01 00:00:00
2010-07-01 00:00:00
2011-01-01 00:00:00
2011-07-01 00:00:00
2012-01-01 00:00:00
2012-07-01 00:00:00
2013-01-01 00:00:00
2013-07-01 00:00:00
2014-01-01 00:00:00
2014-07-01 00:00:00
2015-01-01 00:00:00
2015-07-01 00:00:00
2016-01-01 00:00:00
2016-07-01 00:00:00
2017-01-01 00:00:00
2017-07-01 00:00:00
2018-01-01 00:00:00
2018-07-01 00:00:00
Exportado a PRTU
